<a href="https://colab.research.google.com/github/Pratichhya/Pratichhya.github.io/blob/master/docs/documents/hurricane_dsc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>